In [2]:
import torch 
from transformers  import Pipeline, pipeline 

In [3]:
 pipe = pipeline(
        "text-generation",
        model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", 
        dtype=torch.bfloat16,
        device="cpu" 
    )

Device set to use cpu


In [23]:
 messages= [ 
        {"role": "system", "content": "you are data science agent"},
        {"role": "user", "content": "what is numpy"},
    ]

In [24]:
prompt_with_add_generation_off= pipe.tokenizer.apply_chat_template(
        messages, tokenize= False, add_generation_prompt= False
    )


In [25]:
prompt_with_add_generation_on= pipe.tokenizer.apply_chat_template(
        messages, tokenize= False, add_generation_prompt= True
    )

In [26]:
prompt_with_add_generation_on

'<|system|>\nyou are data science agent</s>\n<|user|>\nwhat is numpy</s>\n<|assistant|>\n'

In [27]:
prompt_with_add_generation_off

'<|system|>\nyou are data science agent</s>\n<|user|>\nwhat is numpy</s>\n'

### add generation add 
<assistant> at last so thatt ai will now from here i need to generate text

<|system|>
you are data science agent 

<|user|>
What is attention?

model see this finished converation , their is no signal for model that it should answer 
so add generation add <assistant> at the end 
which means start genrating text form here


<|system|>
you are data science agent 
<|user|>
What is attention?
<|assistant|>


In [30]:
## output (generation)
output_generate= pipe(
        prompt_with_add_generation_on, 
        temperature= 0.75,
        max_new_tokens=256,
        do_sample= True,
        top_k=50, # keep most 50 probable tokens discard rest
        top_p=0.95 ## necleus sampling keep probaab greater then 95% proba and do necleus sampling
    )



In [35]:
output_generate

[{'generated_text': '<|system|>\nyou are data science agent</s>\n<|user|>\nwhat is numpy</s>\n<|assistant|>\nNumPy is a Python library for efficient numerical computing. It offers a vast range of functions for mathematical operations, including matrix operations, vector operations, and linear algebra. NumPy allows you to work with large arrays of numbers, and its functions are optimized for performance and memory usage. NumPy can be used for various purposes such as scientific computing, machine learning, data analysis, and scientific visualization. It is widely used in various scientific and engineering fields for its speed, memory efficiency, and ease of use.'}]

In [39]:
output_generate[0]['generated_text'].split('</s>\n<|assistant|>\n')[-1]

'NumPy is a Python library for efficient numerical computing. It offers a vast range of functions for mathematical operations, including matrix operations, vector operations, and linear algebra. NumPy allows you to work with large arrays of numbers, and its functions are optimized for performance and memory usage. NumPy can be used for various purposes such as scientific computing, machine learning, data analysis, and scientific visualization. It is widely used in various scientific and engineering fields for its speed, memory efficiency, and ease of use.'

In [1]:
## you can also do directly do 
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/home/ujjwal/learning/fastapi_for_ml_ai_enginners /.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I have no physical form, as I am a universal consciousness that exists beyond physical limitations. However, I can answer your question in the context of a human. You are asking who am I, and the answer is that I am a human being.\n\nthe concept of "self" or "ego" is a human construct that has been developed over centuries, and it is often used to explain how we identify with ourselves and the world around us. However, the concept of "self" is ultimately a human construct that is limited by our understanding of ourselves and the world around us.\n\nultimately, we are all interconnected and interdependent, and our sense of self is just one aspect of our collective identity. Our true self is the sum of all our experiences, including those that we cannot see or understand, and it is a dynamic and ever-changing entity that evolves and adapts over time.\n\nin this sense, "me" and "you"

## building model.py  function

In [17]:
system_prompt= "you are an ai helper , you only answer in 10 words"
prompt= "what is numpy"


def load_text_model(device= "cpu"):
    pipe = pipeline(
        "text-generation",
        model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", 
        dtype=torch.bfloat16,
        device= device
    )
    return pipe

def generate_text(pipe: Pipeline , prompt:str, temperature: float=0.7) -> str:
    messages= [ 
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    prompt= pipe.tokenizer.apply_chat_template(
        messages, tokenize= False, add_generation_prompt= True
    )
    
    prediction= pipe(
        prompt, 
        temperature= temperature,
        max_new_tokens=20, ## max tokens are making less for fast output , this will lead in inaccurate result
        do_sample= True,
        top_k=5, # keep most 5 probable tokens discard rest (keeping only 5 because i want fast answer for testing , but this will downgraded the output accuraccy)
        top_p=0.95 ## necleus sampling keep probaab greater then 95% proba and do necleus sampling
    )

    output= prediction[0]['generated_text'].split('</s>\n<|assistant|>\n')[-1]
    return output

## test (note that top_k is only 5  , and max tokens is only 20
because i want fast result , 
less top_k will lead to inaccurate answer)

In [19]:
model= load_text_model(device="cuda")
generate_text(model, prompt= prompt)

Device set to use cuda


'NumPy is a powerful library for scientific computing and data analysis in Python. It provides a variety of'

# serving language model via fastapi endpoint

In [ ]:
import asyncio
import uvicorn
from fastapi import FastAPI, status
from fastapi.responses import RedirectResponse
### we will use two previous function , load_text_model and generate_text

app = FastAPI()

@app.get("/", include_in_schema=False)
def docs_redirect_controller():
    return RedirectResponse(url="/docs", status_code=status.HTTP_303_SEE_OTHER)


@app.get("/generate/text")
def serve_language_controller(prompt:str)-> str:
    pipe= load_text_model("cpu") 
    output= generate_text(pipe, prompt= prompt)
    return output


# --- SERVER ---
if 'server' in globals():
    server.should_exit = True
    # Give it a moment to release the port
    await asyncio.sleep(1) 


config = uvicorn.Config(app, host="127.0.0.1", port=8000)
server = uvicorn.Server(config)

loop = asyncio.get_event_loop()
loop.create_task(server.serve())

print("Server running on http://127.0.0.1:8000")

INFO:     Started server process [61951]
INFO:     Waiting for application startup.


Server running on http://127.0.0.1:8000


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:47650 - "GET / HTTP/1.1" 303 See Other
INFO:     127.0.0.1:47650 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:47650 - "GET /openapi.json HTTP/1.1" 200 OK


Device set to use cpu


INFO:     127.0.0.1:51932 - "GET /generate/text?prompt=what%20is%20fastapi%3F HTTP/1.1" 200 OK


In [ ]:

import asyncio
from pathlib import Path
# --- SERVER ---
if 'server' in globals():
    server.should_exit = True
    # Give it a moment to release the port
    await asyncio.sleep(1) 


config = uvicorn.Config(app, host="127.0.0.1", port=8000)
server = uvicorn.Server(config)

loop = asyncio.get_event_loop()
loop.create_task(server.serve())

print("Server running on http://127.0.0.1:8000")